In [ ]:
#All the following script and comments have been made in accordance to Merged heart data##

#Importing packages # Make sure that you activate correct environment 
import pandas as pd
import scanpy as sc
import numpy as np
import os
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix, issparse

#Setting up to avoid kernel crashes
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'


sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

#writing output file
results_file = "./outputs/trail/clean_data_scanpy.h5ad"

In [ ]:
#Reading the h5ad file
clean_data = sc.read_h5ad("/Users/srivalli/Desktop/SCA-Uni/Single-cell-data-analysis/Cardiac_cell_analysis/outputs/merged_heart.h5ad")
clean_data

In [ ]:
# Identify duplicated cells based on their annotation information
duplicated_cell_names = clean_data.obs.index[clean_data.obs.duplicated()]

# Print the names of duplicated cells
print("Duplicated cell names:")
print(duplicated_cell_names)

# Identify duplicated cells based on their annotation information
duplicated_cells = clean_data.obs.duplicated()

# Filter out duplicated cells
clean_data = clean_data[~duplicated_cells, :]

# Check the size of the dataset before and after deduplication
print("Size before deduplication:", clean_data.shape)

# Remove any cells with zero genes after deduplication (optional)
clean_data = clean_data[clean_data.obs.n_genes_by_counts > 0, :]

# Check the size of the dataset after removing cells with zero genes
print("Size after deduplication:", clean_data.shape)

In [ ]:
#PREPROCESSING#

#Making var names unique
clean_data.var_names_make_unique()

#Viewing genes that contributes the largest portion in a cell
highest_expr_genes = sc.pl.highest_expr_genes(clean_data)

In [ ]:
sc.pl.violin(clean_data,["n_genes_by_counts", "total_counts", "pct_counts_mt"],jitter=0.4,multi_panel=True)

In [ ]:
#Scatter plots
sc.pl.scatter(clean_data, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(clean_data, x="total_counts", y="n_genes_by_counts")

In [ ]:
#Filtering genes and cells 
sc.pp.filter_cells(clean_data, min_genes=200)
sc.pp.filter_genes(clean_data, min_cells=20)

#Above filtering considers only cells having min 200 genes as a primary criteria and filters the genes which are found in minimum of 20 cells
#CAN BE MODIFIED

In [ ]:
#Normalization using Pearson
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(clean_data)

#Normalizing data matrix using CPM
sc.pp.normalize_total(clean_data,target_sum=1e6)

#Logarithmization of data
sc.pp.log1p(clean_data,base=2)

#Adding to layers
clean_data.layers['normalized_log1p'] = clean_data.X.copy()
clean_data.layers["analytic_pearson_residuals"] = csr_matrix(analytic_pearson["X"])

#Plotting
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
p1 = sns.histplot(clean_data.obs["total_counts"], bins=100, kde=False, ax=axes[0])
axes[0].set_title("Total counts")
p2 = sns.histplot(
    clean_data.layers["analytic_pearson_residuals"].sum(1), bins=100, kde=False, ax=axes[1]
)
axes[1].set_title("Analytic Pearson residuals")
plt.show()

In [ ]:
#Calculating dispersion of highly variable genes
sc.pp.highly_variable_genes(clean_data, layer="scran_normalization")

#Identifying high variable genes satisfying conditions
sc.pp.highly_variable_genes(clean_data, min_mean=0.0125, max_mean=3, min_disp=0.5)
clean_data = sc.pl.highly_variable_genes(clean_data)

In [ ]:
#Writing file
clean_data.write("heart_normalized.h5ad")

In [ ]:
##PRINCIPAL COMPONENT ANALYSIS##

#Reducing dimensions
sc.tl.pca(clean_data, svd_solver="arpack")

#To view the principal components
clean_data.obsm['X_pca']

#Scatter plot for PCA components for visualization 
sc.pl.pca(clean_data)

In [ ]:
#Scatter plot for PCA components for visualization based on coloring of genes
#sc.pl.pca(clean_data,color= "NKG7")

#FIND A GENE

In [ ]:
#Number of PCs to be considered for the data
sc.tl.tsne(clean_data)

#To know the values and count of Principal components
clean_data.obsm['X_pca']

In [ ]:
ensg = clean_data.var_names

# Combine the current working directory with the file name
file_name = "ensembl_gene_ids.txt"
file_path = os.path.join("./outputs/trail/", file_name)


with open(file_path, 'w') as file:
    for ensgid in ensg:
        file.write(ensgid + '\n')

In [ ]:
#Estimates of Principal components contribution to the total variance of the data
sc.pl.pca_variance_ratio(clean_data, log=True)

#Saving results
clean_data.write(results_file)
clean_data

In [ ]:
#COMPUTING NEIGHBOUIRHOOD GRAPH#

#General method
#sc.pp.neighbors(clean_data)
#clean_data
#Can add n_neighbors and n_pca parameters if we would like to consider making clusters based on given params#

#NOT USING THIS

In [ ]:
#Computing neighbors by bbknn - Batch balanced KNN

bbknn_data = sc.external.pp.bbknn(clean_data)

bbknn_data

#We use bbknn only for our analysis

In [ ]:
#EMBEDDING THE NEIGHBOURHOOD GRAPH

#Assinging cells to clusters
sc.tl.louvain(clean_data)

#Partioning data and identifying relationships between clusters
sc.tl.paga(clean_data)
sc.pl.paga(clean_data)

#Data visualization
sc.tl.umap(clean_data)

#Giving colour codes for better visulauization based on genes
sc.pl.umap(clean_data, color=["louvain"])

In [ ]:
#Ranking genes using wilcoxon method
sc.tl.rank_genes_groups(clean_data, "louvain", method="wilcoxon")

#Plotting 
sc.pl.rank_genes_groups(clean_data, n_genes=25, sharey=False)

#Saving data
clean_data.write(results_file)

In [ ]:
#Using logistic regression
sc.tl.rank_genes_groups(clean_data, "louvain", method="logreg", max_iter=100)
sc.pl.rank_genes_groups(clean_data, n_genes=25, sharey=False)

In [ ]:
#To get list of gene names across clusters
gene_clusters = pd.DataFrame(clean_data.uns["rank_genes_groups"]["names"])

gene_clusters.to_csv("./outputs/trail/gene_clusters.csv")
#Can add .head(10) at the end to give output of only 10 hits

In [ ]:
#Getting table with scores and group
clean_data = sc.read(results_file)
result = clean_data.uns["rank_genes_groups"]
groups = result["names"].dtype.names
scores = pd.DataFrame(
    {
        group + "_" + key[:1]: result[key][group]
        for group in groups
        for key in ["names", "pvals"]
    }
)

scores.to_csv("./outputs/trail/scores.csv")
#low p-value indicates that the gene is likely differentially expressed in that cluster compared to others

In [ ]:
#Saving file
clean_data.write(results_file)